In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
import pickle

Using TensorFlow backend.


## Import and curate dataframe

In [4]:
all_dat = pd.read_csv('./curated.csv', index_col=0)

# sift out pwrs
all_dat = all_dat.loc[all_dat['reactor_type'] == 'PWR']  
all_dat = sklearn.utils.shuffle(all_dat)
# only get assemblies with enrichment bigger than 1.5 and bunrup higher than 10,000
all_dat = all_dat.loc[(all_dat['init_enr'] > 1.5) & (all_dat['bu'] > 10000)]


In [7]:
np.max(all_dat['evaluation_date'])

'2014-05-07'

## Categorize Isotopes 

# ANN algorithm ( + hyperparameter grid search)
Below is an attempt to fit prediction of all isotopes in one Artificial Neural Network (ANN).
First, the dataset is fit to an array of potential hyperparameters to find the best combination of hyperparameters for best predicting the isotope composition.


try:
- sigmoid for output layer (currently linear)
- dropout
- 

In [4]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor

In [46]:
x = all_dat[['init_enr', 'bu']].as_matrix()
y = all_dat.iloc[:,5:].as_matrix()

scaler = MinMaxScaler()
xscaler = MinMaxScaler().fit(x)
yscaler = MinMaxScaler().fit(y)
xscale = xscaler.transform(x)
yscale = yscaler.transform(y)

iso_list = list(all_dat.iloc[:, 5:])

/home/teddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/teddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [47]:
model_dict = {}

In [49]:
print(np.shape(xscale))
print(np.shape(yscale))

(104551, 2)
(104551, 61)


In [50]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

# kfold cross validation
kfold = StratifiedKFold(n_splits=3, shuffle=True)
kfold = KFold(n_splits=3)
cvscores = []

for train, test in kfold.split(xscale):
    model = Sequential()
    model.add(Dense(12, input_dim=2, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(len(y[0]), activation='sigmoid'))
    model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae'])
    model.fit(xscale[train], yscale[train], epochs=150, batch_size=50)
    scores = model.evaluate(xscale[test], yscale[test])
    print('%s: %.2f%%' %(model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print('%.2f%% (+/- %.2f%%)' %(np.mean(cvscores), np.std(cvscores)))
model_dict[np.mean(cvscores)] = model

Epoch 1/150
69700/69700 [==============================] - 6s 87us/step - loss: 0.0206 - mean_squared_error: 0.0206 - mean_absolute_error: 0.1070
Epoch 2/150
69700/69700 [==============================] - 5s 68us/step - loss: 0.0032 - mean_squared_error: 0.0032 - mean_absolute_error: 0.0397
Epoch 3/150
69700/69700 [==============================] - 5s 66us/step - loss: 0.0027 - mean_squared_error: 0.0027 - mean_absolute_error: 0.0342
Epoch 4/150
69700/69700 [==============================] - 5s 67us/step - loss: 0.0026 - mean_squared_error: 0.0026 - mean_absolute_error: 0.0330
Epoch 5/150
69700/69700 [==============================] - 6s 85us/step - loss: 0.0024 - mean_squared_error: 0.0024 - mean_absolute_error: 0.0314
Epoch 6/150
69700/69700 [==============================] - 7s 104us/step - loss: 0.0021 - mean_squared_error: 0.0021 - mean_absolute_error: 0.0291
Epoch 7/150
69700/69700 [==============================] - 5s 72us/step - loss: 0.0020 - mean_squared_error: 0.0020 - mean_

69700/69700 [==============================] - 6s 82us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0256
Epoch 112/150
69700/69700 [==============================] - 7s 98us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0257
Epoch 113/150
69700/69700 [==============================] - 6s 85us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0256
Epoch 114/150
69700/69700 [==============================] - 7s 95us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0256
Epoch 115/150
69700/69700 [==============================] - 6s 85us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0256
Epoch 116/150
69700/69700 [==============================] - 7s 105us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0257
Epoch 117/150
69700/69700 [==============================] - 6s 82us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_

Epoch 16/150
69701/69701 [==============================] - 8s 113us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269
Epoch 17/150
69701/69701 [==============================] - 7s 103us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269
Epoch 18/150
69701/69701 [==============================] - 8s 108us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269
Epoch 19/150
69701/69701 [==============================] - 7s 95us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269
Epoch 20/150
69701/69701 [==============================] - 6s 93us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269
Epoch 21/150
69701/69701 [==============================] - 7s 94us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269
Epoch 22/150
69701/69701 [==============================] - 7s 94us/step - loss: 0.0019 - mean_squared_error: 0.001

69701/69701 [==============================] - 4s 62us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269
Epoch 127/150
69701/69701 [==============================] - 6s 82us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269
Epoch 128/150
69701/69701 [==============================] - 5s 69us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269
Epoch 129/150
69701/69701 [==============================] - 5s 72us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269
Epoch 130/150
69701/69701 [==============================] - 5s 65us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269
Epoch 131/150
69701/69701 [==============================] - 5s 66us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269
Epoch 132/150
69701/69701 [==============================] - 5s 74us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_a

69701/69701 [==============================] - 6s 88us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0259
Epoch 87/150
69701/69701 [==============================] - 9s 129us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0259
Epoch 88/150
69701/69701 [==============================] - 5s 75us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0259
Epoch 89/150
69701/69701 [==============================] - 5s 71us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0259
Epoch 90/150
69701/69701 [==============================] - 5s 76us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0259
Epoch 91/150
69701/69701 [==============================] - 9s 129us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0259
Epoch 92/150
69701/69701 [==============================] - 10s 143us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_abs

In [20]:
xtrain, xtest, ytrain, ytest = train_test_split(xscale, yscale)

In [43]:
model = Sequential()
model.add(Dense(12, input_dim=2, kernel_initializer='normal', activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(len(y[0]), activation='sigmoid'))
model.summary()
model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 12)                36        
_________________________________________________________________
dense_19 (Dense)             (None, 4)                 52        
_________________________________________________________________
dense_20 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_21 (Dense)             (None, 61)                305       
Total params: 413
Trainable params: 413
Non-trainable params: 0
_________________________________________________________________


In [44]:
hist = model.fit(xtrain, ytrain, epochs=200, batch_size=50, validation_split=0.2 )

Train on 62730 samples, validate on 15683 samples
Epoch 1/200
62730/62730 [==============================] - 6s 93us/step - loss: 0.0153 - mean_squared_error: 0.0153 - mean_absolute_error: 0.0841 - val_loss: 0.0033 - val_mean_squared_error: 0.0033 - val_mean_absolute_error: 0.0405
Epoch 2/200
62730/62730 [==============================] - 6s 90us/step - loss: 0.0029 - mean_squared_error: 0.0029 - mean_absolute_error: 0.0370 - val_loss: 0.0028 - val_mean_squared_error: 0.0028 - val_mean_absolute_error: 0.0353
Epoch 3/200
62730/62730 [==============================] - 8s 130us/step - loss: 0.0027 - mean_squared_error: 0.0027 - mean_absolute_error: 0.0347 - val_loss: 0.0027 - val_mean_squared_error: 0.0027 - val_mean_absolute_error: 0.0340
Epoch 4/200
62730/62730 [==============================] - 6s 102us/step - loss: 0.0027 - mean_squared_error: 0.0027 - mean_absolute_error: 0.0339 - val_loss: 0.0026 - val_mean_squared_error: 0.0026 - val_mean_absolute_error: 0.0333
Epoch 5/200
62730/62

Epoch 36/200
62730/62730 [==============================] - 5s 81us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0270 - val_loss: 0.0019 - val_mean_squared_error: 0.0019 - val_mean_absolute_error: 0.0269
Epoch 37/200
62730/62730 [==============================] - 4s 70us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0270 - val_loss: 0.0019 - val_mean_squared_error: 0.0019 - val_mean_absolute_error: 0.0268
Epoch 38/200
62730/62730 [==============================] - 4s 68us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0270 - val_loss: 0.0019 - val_mean_squared_error: 0.0019 - val_mean_absolute_error: 0.0270
Epoch 39/200
62730/62730 [==============================] - 4s 68us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0270 - val_loss: 0.0019 - val_mean_squared_error: 0.0019 - val_mean_absolute_error: 0.0268
Epoch 40/200
62730/62730 [==============================] - 5s 73us/

62730/62730 [==============================] - 5s 85us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269 - val_loss: 0.0019 - val_mean_squared_error: 0.0019 - val_mean_absolute_error: 0.0268
Epoch 107/200
62730/62730 [==============================] - 5s 75us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269 - val_loss: 0.0019 - val_mean_squared_error: 0.0019 - val_mean_absolute_error: 0.0269
Epoch 108/200
62730/62730 [==============================] - 5s 79us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269 - val_loss: 0.0019 - val_mean_squared_error: 0.0019 - val_mean_absolute_error: 0.0268
Epoch 109/200
62730/62730 [==============================] - 4s 70us/step - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0269 - val_loss: 0.0019 - val_mean_squared_error: 0.0019 - val_mean_absolute_error: 0.0269
Epoch 110/200
62730/62730 [==============================] - 5s 77us/step - lo

KeyboardInterrupt: 

In [ ]:
print(hist.history.keys())
plt.semilogy(hist.history['loss'])
plt.semilogy(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
enr_bu = np.array([[3.3, 43872]])

xnew = xscaler.transform(enr_bu)
all_dat['err'] = abs(all_dat['bu'] - enr_bu[0][1]) + abs(all_dat['init_enr'] - enr_bu[0][0]) * 1e4
min_err = all_dat['err'].idxmin()
all_dat = all_dat.drop(columns='err')
avg_assem = all_dat.loc[min_err]
print('UDB assembly enr bu: ', [avg_assem['init_enr'], avg_assem['bu']])

ynew = yscaler.inverse_transform(model.predict(xnew))

In [ ]:
print('Isotope \t Predicted \t Data \t %Error')
tot_err = 0
for indx, i in enumerate(iso_list):
    err = ynew[0][indx] - avg_assem[i]
    percent_err = err / avg_assem[i] * 100
    tot_err += err
    print(i, '\t', ynew[0][indx], '\t', avg_assem[i], '\t', percent_err)

print('\n', tot_err)

In [ ]:
model_dict = {}
model_dict[tot_err] = model

In [25]:
model.predict(xnew)

AttributeError: 'KerasRegressor' object has no attribute 'model'

In [20]:
# grid search to find best hyperparameter

def create_model(optimizer='adam', learn_rate=0.01, momentum=0,
                 activation='relu', neurons=1,
                 hidden_layers=1):
    model = Sequential()
    model.add(Dense(neurons, input_dim=2, activation=activation))
    # consider putting in dropout?
    for i in range(hidden_layers):
        model.add(Dense(neurons, activation=activation))
    model.add(Dense(60, activation='linear'))
    model.compile(loss='mean_absolute_percentage_error', optimizer=optimizer,
                  metrics=['mean_absolute_percentage_error'])
    return model

x = all_dat[['init_enr', 'bu']].as_matrix()
y = all_dat.iloc[:,4:]

model = KerasRegressor(build_fn=create_model, verbose=0)
param_grid = dict(batch_size=[10, 20, 40, 60, 80],
                  epochs=[10, 50, 100, 200],
                  optimizer=['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
                  # learn rate and momentum implementation???
                  activation = ['relu', 'softmax', 'softplus', 'softsign', 'tanh', 'linear'],
                  neurons = [1, 5, 10, 20, 40, 80],
                  hidden_layers = [0, 1, 3, 5, 10]
                  )
result_dict = {}

# for every hyperparameter, run a grid search
for param, param_list in param_grid.items():
    print(param, '\n\n')
    use = {param: param_list}
    grid = GridSearchCV(estimator=model, param_grid=use)
    grid_result = grid.fit(x, y)
    result_dict[param] = (grid_result.best_score_, grid_result.best_params_)
    # summarize results
    print('Best: %f using %s' %(grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']

    #for mean, stdev. param in zip(means, stds, params):
    #    print('%f (%f) with: %r' %(mean, stdev, param))


/home/teddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()
/home/teddy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


batch_size 




/home/teddy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Error when checking target: expected dense_6 to have shape (60,) but got array with shape (62,)

In [19]:
result_dict

{'batch_size': (-37339.71153245486, {'batch_size': 20}),
 'epochs': (-26118.439008011042, {'epochs': 100}),
 'optimizer': (-64947.83971664397, {'optimizer': 'Adamax'}),
 'activation': (-54298.19828140842, {'activation': 'softsign'}),
 'neurons': (-72339.40957284004, {'neurons': 1}),
 'hidden_layers': (-28876.054231980685, {'hidden_layers': 10})}

In [20]:
# use the best:
def get_val(dictionary, key):
    return dictionary[key][1][key]
batch_size = get_val(result_dict, 'batch_size')
epochs = get_val(result_dict, 'epochs')
optimizer = get_val(result_dict, 'optimizer')
activation = get_val(result_dict, 'activation')
neurons = get_val(result_dict, 'neurons')
hidden_layer = get_val(result_dict, 'hidden_layers')
def create_model(optimizer='adam', learn_rate=0.01, momentum=0,
                 activation='relu', neurons=1,
                 hidden_layers=1):
    model = Sequential()
    model.add(Dense(neurons, input_dim=2, activation=activation))
    # consider putting in dropout?
    for i in range(hidden_layers):
        model.add(Dense(neurons, activation=activation))
    model.add(Dense(60, activation='linear'))
    model.compile(loss='mean_absolute_percentage_error', optimizer=optimizer,
                  metrics=['mean_absolute_percentage_error'])
    return model

x = all_dat[['init_enr', 'bu']].as_matrix()
y = all_dat.iloc[:,4:]

model =  create_model(optimizer=optimizer, activation=activation, neurons=neurons,
                     hidden_layers=hidden_layer)
print(model)
model.fit(x, y, epochs=epochs, batch_size=batch_size, validation_split=0.3)
model.evaluate


/home/teddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Train on 73185 samples, validate on 31366 samples
Epoch 1/100
73185/73185 [==============================] - 24s 334us/step - loss: 88177.8860 - mean_absolute_percentage_error: 88177.8860 - val_loss: 76297.6464 - val_mean_absolute_percentage_error: 76297.6464
Epoch 2/100
73185/73185 [==============================] - 18s 243us/step - loss: 71776.8624 - mean_absolute_percentage_error: 71776.8624 - val_loss: 66387.5748 - val_mean_absolute_percentage_error: 66387.5748
Epoch 3/100
73185/73185 [==============================] - 17s 227us/step - loss: 71383.8011 - mean_absolute_percentage_error: 71383.8011 - val_loss: 86997.0727 - val_mean_absolute_percentage_error: 86997.0727
Epoch 4/100
73185/73185 [==============================] - 16s 218us/step - loss: 72922.7693 - mean_absolute_percentage_error: 72922.7693 - val_loss: 66281.3289 - val_mean_absolute_percentage_error: 66281.3289
Epoch 5/100
73185/73185 [==============================] - 16s 217us/step - loss: 75174.1389 - mean_absolute_p

Epoch 78/100
73185/73185 [==============================] - 15s 208us/step - loss: 70375.6989 - mean_absolute_percentage_error: 70375.6989 - val_loss: 74180.0553 - val_mean_absolute_percentage_error: 74180.0553
Epoch 79/100
73185/73185 [==============================] - 16s 212us/step - loss: 70814.4295 - mean_absolute_percentage_error: 70814.4295 - val_loss: 85027.3690 - val_mean_absolute_percentage_error: 85027.3690
Epoch 80/100
73185/73185 [==============================] - 15s 211us/step - loss: 70454.8263 - mean_absolute_percentage_error: 70454.8263 - val_loss: 86863.8476 - val_mean_absolute_percentage_error: 86863.8476
Epoch 81/100
73185/73185 [==============================] - 15s 210us/step - loss: 70866.0745 - mean_absolute_percentage_error: 70866.0745 - val_loss: 90682.0947 - val_mean_absolute_percentage_error: 90682.0947
Epoch 82/100
73185/73185 [==============================] - 15s 210us/step - loss: 69912.6575 - mean_absolute_percentage_error: 69912.6575 - val_loss: 52640

<bound method Sequential.evaluate of <keras.models.Sequential object at 0x7f0e9c2dd710>>

## Test the trained model

In [ ]:
testx = x[300]
prediction = model.predict(x)
iso_list = list(y)

In [ ]:
err = []

for indx, val in enumerate(prediction[300]):
    err.append((val-np.array(y.iloc[300])[indx]) / np.array(y.iloc[300])[indx])
    print(iso_list[indx], '\t\t', val, '\t\t', np.array(y.iloc[300])[indx])

print('\n\n\n')
    
for indx, val in enumerate(err):
    print(iso_list[indx], '\t', err[indx])